В этом примере будет показана работа с объектом класса *InverseSolver* программной библиотеки *SFCPurifier*.
Класс *InverseSolver* предназначен для решения обратной задачи светорассеяния методом поиска ближайшего элемента в базе данных и алгоритмами глобальной оптимизации (в разработке). Поиск ближайшего элемента может производится несколькими методами: 
- полный перебор (bruteforce)
- kd-дерево (kd-tree)
- метрическое дерево (ball-tree).

Для оценки математического ожиданя и дисперсии найденных параметров возможно использование байесовского подхода.

In [1]:
import pandas as pd
import numpy as np
import SFCPurifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [2]:
solver = SFCPurifier.InverseSolver()
# binn = pocket.Purifier()
# binn.openBinFile(r'C:\Users\Kamov\Desktop\20240513\lat_3.7\3D_pockets_uint16.bin')

In [3]:
mon_path = r'C:\Users\Kamov\Desktop\Новая папка\PLT\Plt_200k_h2.csv'
mon_dim = r'C:\Users\Kamov\Desktop\Новая папка\PLT\dimer_bd_5000.csv'

In [4]:
df_mon = pd.read_csv(mon_path, delimiter=';')
df_dim = pd.read_csv(mon_dim, delimiter=';')

mon_par_labl = df_mon.columns[:4]
dim_par_labl = df_dim.columns[:11]

angles_range = df_mon.columns[4:]

angles_forw = angles_range[10:70]
angles_back = angles_range[90:180]

m_forw = solver.weightM(np.array(angles_forw, dtype=int))
m_back = solver.weightM(np.array(angles_back, dtype=int))


X_mon_forw, X_mon_back = np.array(df_mon[angles_forw]) * m_forw, np.array(df_mon[angles_back]) * m_back
Y_mon = np.array(df_mon[mon_par_labl])
X_dim_forw,X_dim_back = np.array(df_dim[angles_forw]) * m_forw, np.array(df_dim[angles_back]) * m_back
Y_dim = np.array(df_dim[dim_par_labl])

In [5]:
solver.initModels(X_mon_forw, Y_mon)
res = solver.NearestNeighborsRegressorFit(X_dim_forw, model='BruteForce')
mean, sigmas = solver.BayessianEstimation(X_dim_forw, neighbors=1000)

100%|██████████| 5000/5000 [03:26<00:00, 24.19it/s]


In [6]:
np.savetxt('nearest.txt', res)
np.savetxt('means.txt', mean)
np.savetxt('sigmas.txt', sigmas)